In [1]:
!cp -R /content/drive/MyDrive/Digikala-comments /content
%cd /content/drive/MyDrive/Digikala-comments
!tree

/content/drive/MyDrive/Digikala-comments
/bin/bash: tree: command not found


### Reading Data

In [2]:
import pandas as pd

df = pd.read_csv('/content/Digikala-comments/Data/Data.csv')
df.head()

,id,title,comment,rate,verification_status
0,0,کیفیت و حجم صدای عااااالی,این محصول توی بازار اصلاااا پیدا نمیشه من کل ت...,100.0,0
1,1,شش ماه مصرف!!,دوبار از این مدل گرفتم اولاش خوبه ولی بعد از ش...,5.0,0
2,2,کارآیی,به نظر من فقط برای کارای سبک و دیدن فیلم و مطا...,60.0,0
3,3,بررسی کمی و کیفی,برای من بسیار مناسب و خریدش در شگفت انگیز حتما...,0.0,0
4,4,بسته بندی ضعیف,ظاهر بامزه ای داره ولی عکسش شبیه خودش نیست جنس...,60.0,0


### Cleaning Data

In [3]:
# Null Values
df = df[~((df.title.isnull()) & (df.comment.isnull()))]
df = df.reset_index(drop=True)

In [4]:
# Data Types
def set_types(df):
    df.title = df.title.astype(str)
    df.comment = df.comment.astype(str)
    df.rate = df.rate.astype('int')
    return df
df = set_types(df)

In [5]:
# Managing NaNs
def replace_nan(entry):
    if entry == 'nan':
        return '#'
    return entry
df.title = df.title.apply(replace_nan)
df.comment = df.comment.apply(replace_nan)

In [6]:
# Making our sentiment colmn
df['Phrase'] = df['title'] + ' ' + df['comment']
df['Sentiment'] = df['rate']
df.drop(columns=['title', 'verification_status', 'comment', 'rate'], inplace=True) 
df.head(1)

,id,Phrase,Sentiment
0,0,کیفیت و حجم صدای عااااالی این محصول توی بازار ...,100


In [7]:
# Duplicates
df.drop_duplicates(subset =['Phrase'], inplace = True)

#### more cleaning

In [10]:
import numpy as np
from hazm import word_tokenize, stopwords_list, InformalLemmatizer
import re
lemma = InformalLemmatizer()

# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

# https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py  emoticons list
# https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt Chat shortcuts

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r' ', text)

def remove_punctuations(text):
    punctuations = re.compile(r'[~`!@#$%^&*(,<،>){}\\/|\'"?؟_+-=~\[\]]')
    return punctuations.sub(r' ', text)

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r' ', text)

def remove_weird_chars(text):
    weridPatterns = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               u"\u2069"
                               u"\u2066"
                               u"\u200c"
                               u"\u2068"
                               u"\u2067"
                               "]+", flags=re.UNICODE)
    patterns = [re.compile('\r'), re.compile('\n'), re.compile('&amp;')]
    text = weridPatterns.sub(r'', text)
    for p in patterns:
        text = p.sub(r' ', text)
    return text

def remove_extra_repeated_alpha(text):
    """
    Remove extra repeated alphabets in a word
    check these links:
    demo : https://regex101.com/r/ALxocA/1
    Question: https://bit.ly/2DoiPqS
    """
    return re.sub(r'([^\W\d_])\1{2,}', r'\1', text)


def clean_up(text, url=True, html=True, weird_patterns=True , lemmatize=False, 
               stopwords=True, isalpha=False, punctuations=True, remove_extra_alpha=True):
    # remove url
    if url:
        text = remove_urls(text)
    # remove html tags
    if html:
        text = remove_html(text)
    # remove emokis / symbols & pictographs / transport & map symbols / flags (iOS)
    if weird_patterns:
        text = remove_weird_chars(text)
    # remove punctuations
    if punctuations:
        text = remove_punctuations(text)
    # Alter words with repeated alphabets
    if remove_extra_repeated_alpha:
        text = remove_extra_repeated_alpha(text)
    # tokenize text
    tokens = word_tokenize(text)
    # remove stop words
    if stopwords:
        tokens = [word for word in tokens if word not in stopwords_list()]
    # remove non-alphabetic items
    if isalpha:
        tokens = [word for word in tokens if word.isalpha()]
    # lemmatize words
    if lemmatize:
        tokens = [lemma.lemmatize(word) for word in tokens]
    text = ' '.join(tokens)
    
    return text



In [11]:
# not cleaned example
test = df.loc[502].Phrase
test

'دوستان عزیز لطفا بعد چندروز استفاده از S7Edge نظر خود را بنویسید دوستان سلام . اول از همه می خواستم خواهش کنم برای اینکه اطلاعات درست به همگان برسد لطفا دوستانی نظر دهند که  کالا رو تهیه کرده وچند روز استفاده کردنند و بتوانند اطلاعات واقعی درک شده رو اعلام کنند. واز هرگونه تخیل و افکار و انتظار غیر منطقی خوداری کنند تا فردی که تصمیم به خرید دارد بتواند درست انتخاب کند. با تشکر از هممیهنان عزیز که برای خود ودیگران احترام قائل هستند.\nمن حدود 2ماه است از این گوشی سامسونگ گالکسی  اس هفت ایج استفاده کرده و موارد کاملا واقعی که درک ودیدم و در حال حاضر مشکلات ان را احساس میکنم را دراختیار دوستان میگذارم تا بهتر بتوانند تصمیم بگیرند.\nSamsung Galaxy S7 edge Dual  935FD . بهترین گوشی جهان تا این تاریخ 1395/12/03  از نظر قیمت نصبت به آیفون {سیب گاز زده}. بهترین طراحی. شکیل بودن. سرعت و قدرت عالی حتی از آیفون 7 و دیگر دستگاههای موجود. روانی در نرم افزارها و بخصوص گیم اسفالت که اخر بازی با گرافیک سه بعدی عالی است. قدرت ران چندین نرم افزار با هم بدون هنگی.کیفیت تصویر نمایش که دیگه عالیه با سوپر ا

In [12]:
clean_up(test)

'دوستان عزیز لطفا چندروز استفاده S Edge بنویسید دوستان سلام خواستم خواهش اطلاعات درست همگان برسد لطفا دوستانی کالا وچند روز استفاده کردنند بتوانند اطلاعات واقعی درک اعلام واز هرگونه تخیل افکار انتظار منطقی خوداری تصمیم خرید بتواند درست انتخاب تشکر هممیهنان عزیز ودیگران احترام قائل ماه گوشی سامسونگ گالکسی اس هفت ایج استفاده موارد واقعی درک ودیدم حاضر مشکلات ان احساس میکنم دراختیار دوستان میگذارم بتوانند تصمیم بگیرند Samsung Galaxy S edge Dual FD گوشی جهان تاریخ قیمت نصبت آیفون سیب گاز زده طراحی شکیل سرعت قدرت آیفون دستگاههای موجود روانی نرم افزارها بخصوص گیم اسفالت اخر بازی گرافیک بعدی قدرت ران چندین نرم افزار هنگی کیفیت تصویر نمایش دیگه عالیه سوپر امو ال ای دی دوربین واقعی مگاپیکسل وبهتر اینه مشخصات سایت سامسونگ انگلیسی ببینیدچون سایتهای فارسی گاها اشتباهی میزنند دستگاه پرقدرتی حتما پردازش سریع هسته نتیجتا مصرف انرژی تکرار میکنم ایراد دستگاه باطری میلی امپر ان میلی امپر میگذاشتن استفاده اینترنت شبکه های مجازی موسیقی گیم دارید ان استفاده پاور بانک میلی امپر اصلی سامسونگ دستگاه شارژ میکن

In [13]:
# Applying our function to Phrases
# may take some time
# df['Phrase'] = df['Phrase'].apply(clean_up)

In [14]:
# Saving the cleaned data
# df.to_csv('/content/drive/MyDrive/Digikala-comments/Data/Cleaned-data.csv')

In [15]:
# Distribution of sentiment classes across our data.
df = pd.read_csv('/content/drive/MyDrive/Digikala-comments/Data/Cleaned-data.csv')
df.head()

,Unnamed: 0,id,Phrase,Sentiment
0,0,0,کیفیت حجم صدای محصول توی بازار اصلا نمیشه تهرا...,100
1,1,1,ماه مصرف دوبار مدل گرفتم اولاش خوبه ماه باد می...,5
2,2,2,کارآیی کارای سبک دیدن فیلم مطالعه مناسبه,60
3,3,3,بررسی کیفی خریدش شگفت انگیز حتما پیشنهاد,0
4,4,4,بسته ضعیف ظاهر بامزه ای داره عکسش شبیه جنس چوب...,60


### Tokenizing our data

We will be tokenizing this text to create two input tensors; our input IDs, and attention mask.

We will contain our tensors within two numpy arrays, which will be of dimensions len(df) * 512 - the 512 is the sequence length of our tokenized sequences for BERT, and len(df) the number of samples in our dataset.

In [16]:
seq_len = 512
num_samples = len(df)

num_samples, seq_len

(179348, 512)

Tokenizing time! using BertTokenizer.

In [22]:
df['Phrase'] = df['Phrase'].astype('str')

In [ ]:
from transformers import AutoTokenizer

# initialize tokenizer
model_name = "HooshvareLab/bert-fa-zwnj-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# tokenize - this time returning Numpy tensors
tokens = tokenizer(df['Phrase'].tolist(), max_length=seq_len, truncation=True,
                   padding='max_length', add_special_tokens=True,
                   return_tensors='np')

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
# We have three numpy arrays - input_ids, token_type_ids, and attention_mask.
tokens.keys()

In [ ]:
tokens['input_ids'][:10], tokens['attention_mask'][:10]


Now is the time for saving our Numpy binary files

In [ ]:
with open('movie-xids.npy', 'wb') as f:
    np.save(f, tokens['input_ids'])
with open('movie-xmask.npy', 'wb') as f:
    np.save(f, tokens['attention_mask'])

In [ ]:
# deliting tokens to free up memory.
del tokens

Our job with Phrases is done and now is the time to move to our Sentiments. currently, our target data is a set of integer values in the Sentiment column of our data frame `df` . We need to extract these values and *one-hot*
 encode them into another NumPy array, which will have the dimensions `len(df) * number of label classes`
.

In [ ]:
# first extract sentiment column
arr = df['Sentiment'].values

In [ ]:
# we then initialize the zero array
labels = np.zeros((num_samples, arr.max()+1))
labels.shape

We are able to use `arr.max()+1` to define our second dimension here because we have the values `[0, 1, 2, 3, 4]` in our Sentiment column, there are five unique labels which means we need our labels array to have five columns (one for each) - `arr.max() = 4`, so we do `4 + 1` to get our required value of `5`.

Now we use the current values in our `arr` of `[0, 1, 2, 3, 4]` to place `1` values in the correct positions of our presently zeros-only array:

In [ ]:
labels[np.arange(num_samples), arr] = 1

labels

In [ ]:
with open('movie-labels.npy', 'wb') as f:
    np.save(f, labels)

In [ ]:
afs